## Part 4: Fraud

In [1]:
%run "../scripts/ETL.py"

22/10/03 23:06:02 WARN Utils: Your hostname, Aishwaryas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.6.81 instead (on interface en0)
22/10/03 23:06:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 23:06:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


AnalysisException: Cannot resolve column name "postcodes" among (id, postcode, locality, state, long, lat, dc, type, status, sa3, sa3name, sa4, sa4name, region, Lat_precise, Long_precise, SA1_MAINCODE_2011, SA1_MAINCODE_2016, SA2_MAINCODE_2016, SA2_NAME_2016, SA3_CODE_2016, SA3_NAME_2016, SA4_CODE_2016, SA4_NAME_2016, RA_2011, RA_2016, MMM_2015, MMM_2019, ced, altitude, chargezone, phn_code, phn_name, lgaregion, electorate, electoraterating)

The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [2]:
print(fraud_consumer.count())
fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
fraud_consumer.limit(5)

34864


user_id_1,order_datetime_1,cons_fraud_prob
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035


Probability of fraud for high revenue merchants...?

In [3]:
print(fraud_merchants.count())
fraud_merchants = fraud_merchants.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
fraud_merchants.limit(5)

114


merchant_abn_1,order_datetime,merch_fraud_prob
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [4]:

merchant_data = final_join.select("merchant_abn", "categories","revenue_levels","dollar_value","user_id","consumer_id")
merchant_data.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id
10023283211,"furniture, home f...",e,69.90225010580168,1859,797884
10023283211,"furniture, home f...",e,277.5959492940812,19083,1250662
10023283211,"furniture, home f...",e,218.0471981318539,14673,349420
10023283211,"furniture, home f...",e,429.13905563051316,10932,1412788
10023283211,"furniture, home f...",e,105.48434125775125,18249,1314551


- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [5]:
#Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

#Joining consumer fraud and merchant data together

merchant_data.createOrReplaceTempView("temp")

fraud_consumer.createOrReplaceTempView("temp2")

join_1 = spark.sql(""" 

SELECT *
FROM temp

INNER JOIN temp2

ON temp.user_id = temp2.user_id_1 
""")
join_1.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob
11237511112,"opticians, optica...",c,86.43306925925785,15,1463076,15,2021-11-27,16.99959017524405
21439773999,"cable, satellite,...",a,110.0283925711613,18569,1417071,18569,2021-12-22,9.420406825263207
46298404088,"books, periodical...",a,63.71506278687261,18651,1362176,18651,2021-09-15,11.241168015565329
46298404088,"books, periodical...",a,63.71506278687261,18651,1362176,18651,2022-01-22,14.042996949112718
46298404088,"books, periodical...",a,63.71506278687261,18651,1362176,18651,2022-02-11,14.871507132407407


In [6]:
#Joining join_1 to merchant fraud 

#Joining all test + predicting Sets Together 
join_1.createOrReplaceTempView("temp")

fraud_merchants.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


INNER JOIN temp2

ON temp.merchant_abn = temp2.merchant_abn_1
""")
final_join.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob
21439773999,"cable, satellite,...",a,110.0283925711613,18569,1417071,18569,2021-12-22,9.420406825263207,21439773999,2021-11-26,28.504479048104585
96680767841,motor vehicle sup...,a,244.19185320012286,20924,1051408,20924,2022-02-17,9.76740823700797,96680767841,2021-11-26,29.555244690425017
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-02-10,11.94169979516484,18158387243,2021-11-29,28.956947892226466
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-02-10,11.94169979516484,18158387243,2021-11-26,29.102418916409913
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-01-14,13.321204535044247,18158387243,2021-11-29,28.956947892226466


In [14]:
#Changing all null probability values to 0.01: 

MIN_P = 0.01
final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [8]:
#Selecting columns for model: 

model_data = final_join.select("cons_fraud_prob", "merch_fraud_prob")

In [35]:
#Changing data types from string to float: 
from pyspark.sql.types import DoubleType
model_data = model_data.withColumn("cons_fraud_prob",col("cons_fraud_prob").cast(DoubleType())) \
.withColumn("merch_fraud_prob",col("merch_fraud_prob").cast(DoubleType()))

In [36]:
# Vectorizing 
from pyspark.ml.feature import VectorAssembler
assembler1 = VectorAssembler(
inputCols=["cons_fraud_prob", "merch_fraud_prob"],
outputCol= "features")

transformed_model = assembler1.transform(model_data)

In [46]:
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.linalg import Vectors

gmm = GaussianMixture().setK(3).setSeed(14)
model = gmm.fit(transformed_model)

print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)

Gaussians shown as a DataFrame: 
+---------------------------------------+--------------------------------------------------------------------------------------+
|mean                                   |cov                                                                                   |
+---------------------------------------+--------------------------------------------------------------------------------------+
|[15.748566641442878,38.748537873949616]|78.85396613365751   0.5585279507501096  \n0.5585279507501096  144.90762629222507      |
|[27.926429462440662,29.173328490648014]|217.58438955775452  0.3747015977880894  \n0.3747015977880894  1.1570800189030341      |
|[11.444691905180804,29.159257822363525]|6.9975005509476915   0.08279856656877138  \n0.08279856656877138  0.8943562927790535   |
+---------------------------------------+--------------------------------------------------------------------------------------+



In [50]:
centers = model.getPredictionCol()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
p
r
e
d
i
c
t
i
o
n


In [42]:
model.gaussiansDF.limit(5)

mean,cov
[15.7485666414428...,78.85396613365751...
[27.9264294624406...,217.5843895577545...
[11.4446919051808...,6.997500550947691...
